# KI-ENNA: (E)in (N)euronales (N)etz zum (A)usprobieren
Live-Demo basierend auf KI-ENNA 2.0.1 von Prof. Dr. habil. Dennis Klinkhammer

![title](https://github.com/statistical-thinking/ki-enna-demo/blob/main/content/img/image1.png?raw=true)
## Aufgabenstellung
Die obere Darstellung zeigt drei **verschiedene Pflanzenarten**, deren **Blattlänge und Blattbreite** gemessen worden sind. Während die mit Kreisen dargestellte Planzenart sich eindeutig von den beiden anderen Pflanzenarten unterscheidet, weisen von den anderen Pflanzenarten **einzelne Pflanzen die gleiche Blattlänge und Blattbreite** auf. Daher sind diese Pflanzenarten **schwer zu unterscheiden**. Ein neuronales Netzwerk kann bei der Unterscheidung der ähnlichen Pflanzenarten hilfreich sein.

![title](https://github.com/statistical-thinking/ki-enna-demo/blob/main/content/img/image2.png?raw=true)
## Training eines neuronalen Netzwerks
Die interaktive Benutzeroberfläche ermöglicht es Dir, in nur wenigen Schritten Dein eigenes neuronales Netzwerk als **Grundlage einer echten Künstlichen Intelligenz zu erstellen und zu trainieren**. Dabei kannst Du die Anzahl an Schichten, Neuronen und Epochen, aber auch die Lernrate selbst einstellen. Neuronale Netzwerke sind wie kleine Denkmaschinen, **die in Schichten arbeitet**. In die **Eingabeschicht** können Zahlen eingegeben werden, die in den **versteckten Schichten** weiter verarbeitet werden. Am Ende gibt dann die **Ausgabeschicht** eine Antwort. **Neuronen treiben die Denkmaschine an**. Sie rechnen mit den Zahlen und **schicken Zwischenergebnisse an Neuronen in anderen Schichten weiter**. Aber Achtung: Nicht immer führen mehr Schichten und Neuronen zu einem besseren Ergebnis. Damit das funktioniert, braucht ein neuronales Netzwerk Wiederholungen: Eine **Epoche ist ein Durchgang durch alle Übungsaufgaben**. Das macht das neuronale Netzwerk mehrmals, um immer besser zu werden. Die **Lernrate bestimmt, wie schnell das neuronale Netzwerk dabei seine Fehler verbessert**: Ist die Lernrate zu hoch, lernt es zu schnell und macht vielleicht neue Fehler.

In [ ]:
# --- Erster Code ---
import micropip
await micropip.install('ipywidgets')
import random, math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Dataset ---
Xtest = [[ 0.81575475, -0.21746808, -0.12904165, -0.65303909],
         [ 0.05761837,  1.59476592,  0.84485761,  1.71304456],
         [ 0.96738203,  0.68864892, -0.00730424, -0.41643072],
         [ 2.02877297,  0.38660992,  2.06223168,  1.00321947],
         [ 1.42226386,  0.99068792,  1.33180724,  0.29339437],
         [ 0.81575475,  0.99068792,  1.21006983,  1.4764362 ],
         [-1.00377258,  0.38660992, -0.49425387, -0.41643072],
         [ 0.05761837, -0.51950708, -0.00730424,  0.29339437],
         [ 0.36087292,  0.38660992,  1.08833242,  1.23982783],
         [ 0.66412748,  0.38660992,  0.35790798,  1.4764362 ],
         [ 0.05761837,  0.08457092,  0.84485761,  0.29339437],
         [-0.70051802, -0.51950708,  0.23617057,  0.53000274],
         [ 0.20924564, -0.21746808,  0.84485761,  1.00321947],
         [-0.24563619,  0.08457092, -0.25077906, -0.65303909],
         [-2.06516352, -1.42562408, -1.95510276, -1.59947255],
         [-1.15539985, -1.42562408, -1.34641572, -1.36286418],
         [ 0.05761837, -1.12358508, -0.00730424, -0.41643072],
         [ 0.20924564,  0.08457092, -0.73772869, -0.88964745],
         [-0.39726347, -0.51950708,  0.23617057, -0.17982236],
         [ 0.5125002 ,  0.08457092, -0.37251647, -0.88964745]]

ytrue = [0,1,0,1,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0]

# --- Activation & Loss ---
def sigmoid(x): return 1 / (1 + math.exp(-x))
def sigmoid_derivative(out): return out * (1 - out)
def relu(x): return max(0, x)
def relu_derivative(out): return 1 if out > 0 else 0
def binary_cross_entropy(pred, y): 
    epsilon = 1e-7
    return - (y * math.log(pred + epsilon) + (1 - y) * math.log(1 - pred + epsilon))

def binary_cross_entropy_derivative(pred, y): 
    epsilon = 1e-7
    return -(y / (pred + epsilon)) + (1 - y) / (1 - pred + epsilon)

# --- Dense Layer ---
def dense_forward(x, w, b, act='relu'):
    outs, pres = [], []
    for j in range(len(w)):
        z = sum(x[i] * w[j][i] for i in range(len(x))) + b[j]
        pres.append(z)
        outs.append(sigmoid(z) if act == 'sigmoid' else relu(z))
    return outs, pres

def dense_backward(x, grad_out, out, pre, w, b, act='relu', lr=0.01):
    grad_in = [0] * len(x)
    for j in range(len(w)):
        delta = grad_out[j] * (sigmoid_derivative(out[j]) if act == 'sigmoid' else relu_derivative(pre[j]))
        for i in range(len(x)):
            grad_in[i] += w[j][i] * delta
            w[j][i] -= lr * delta * x[i]
        b[j] -= lr * delta
    return grad_in

# --- Visualisierung ---
def plot_network_architecture(layer_sizes, input_dim):
    all_layers = [input_dim] + layer_sizes + [1]
    fig, ax = plt.subplots(figsize=(max(8, len(all_layers) * 1.5), 5))
    ax.axis('off')
    v_spacing = 1.5
    h_spacing = 2

    for i, n_neurons in enumerate(all_layers):
        layer_x = i * h_spacing
        layer_y_start = -(n_neurons - 1) * v_spacing / 2
        for j in range(n_neurons):
            circle = plt.Circle((layer_x, layer_y_start + j * v_spacing), 0.3, color='black', ec='black')
            ax.add_patch(circle)
            if i == 0:
                ax.text(layer_x - 0.6, layer_y_start + j * v_spacing, f'x{j+1}', ha='center', va='center')
            elif i == len(all_layers)-1:
                ax.text(layer_x + 0.6, layer_y_start + j * v_spacing, f'ŷ', ha='center', va='center')
        if i > 0:
            prev_neurons = all_layers[i-1]
            for pj in range(prev_neurons):
                for cj in range(n_neurons):
                    x1 = (i-1) * h_spacing
                    y1 = -(prev_neurons - 1) * v_spacing / 2 + pj * v_spacing
                    x2 = i * h_spacing
                    y2 = -(n_neurons - 1) * v_spacing / 2 + cj * v_spacing
                    ax.plot([x1, x2], [y1, y2], color='gray', lw=0.5)
    ax.set_xlim(-1, len(all_layers) * h_spacing)
    ax.set_ylim(-6, 6)
    plt.show()

# --- Speicher für Evaluation ---
trained_model = {}

# --- Training ---
def train_model(X, y, layer_sizes, epochs, lr):
    dims = [len(X[0])] + layer_sizes + [1]
    weights, biases = [], []
    for i in range(len(dims)-1):
        w = [[random.uniform(-0.5, 0.5) for _ in range(dims[i])] for _ in range(dims[i+1])]
        b = [random.uniform(-0.5, 0.5) for _ in range(dims[i+1])]
        weights.append(w)
        biases.append(b)

    loss_trace = []
    for epoch in range(epochs):
        total_loss = 0
        for xi, yi in zip(X, y):
            x = xi
            acts, pres = [], []
            for i in range(len(weights)):
                act = 'sigmoid' if i == len(weights)-1 else 'relu'
                x, pre = dense_forward(x, weights[i], biases[i], act)
                acts.append(x)
                pres.append(pre)
            loss = binary_cross_entropy(acts[-1][0], yi)
            total_loss += loss
            grad = [binary_cross_entropy_derivative(acts[-1][0], yi)]
            for i in reversed(range(len(weights))):
                act = 'sigmoid' if i == len(weights)-1 else 'relu'
                inp = xi if i == 0 else acts[i-1]
                grad = dense_backward(inp, grad, acts[i], pres[i], weights[i], biases[i], act, lr)
        loss_trace.append(total_loss)

    return {
        "weights": weights,
        "biases": biases,
        "loss_trace": loss_trace
    }

# --- Widgets ---
layers_slider = widgets.IntSlider(value=2, min=1, max=5, step=1, description='Layeranzahl:')
neuron_sliders_box = widgets.VBox()
epochs_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Epochen:')
lr_slider = widgets.FloatSlider(value=0.05, min=0.001, max=1.0, step=0.01, description='Lernrate:')
train_button = widgets.Button(description="Trainieren")
train_output = widgets.Output()

# --- Neuronenslider aktualisieren ---
def update_neuron_sliders(*args):
    count = layers_slider.value
    sliders = []
    for i in range(count):
        sliders.append(widgets.IntSlider(value=4, min=1, max=10, step=1, description=f'Neuronen {i+1}:'))
    neuron_sliders_box.children = sliders

layers_slider.observe(update_neuron_sliders, names='value')
update_neuron_sliders()

# --- Callback für Training & Visualisierung ---
def on_train_click(b):
    train_output.clear_output()
    layer_sizes = [s.value for s in neuron_sliders_box.children]
    epochs = epochs_slider.value
    lr = lr_slider.value

    model = train_model(Xtest, ytrue, layer_sizes, epochs, lr)
    trained_model.clear()
    trained_model.update({
        "weights": model["weights"],
        "biases": model["biases"],
        "loss_trace": model["loss_trace"],
        "layer_sizes": layer_sizes
    })

    with train_output:
#       print("Trainiertes Netzwerk:")
        plot_network_architecture(layer_sizes, input_dim=len(Xtest[0]))

train_button.on_click(on_train_click)

# --- Anzeige ---
display(widgets.VBox([
    layers_slider,
    neuron_sliders_box,
    epochs_slider,
    lr_slider,
    train_button,
    train_output
]))

![title](https://github.com/statistical-thinking/ki-enna-demo/blob/main/content/img/image3.png?raw=true)
## Bewertung eines neuronalen Netzwerks
Die **Verlust-Funktion ist wie ein Schiedsrichter**. Sie schaut sich die Antworten des neuronalen Netzwerks an und entscheidet, ob das Ergebnis richtig oder falsch ist. Wenn ein neuronales Netzwerk falsch liegt, zeigt die Verlust-Funktion **wie weit die Antwort von der richtigen Lösung entfernt ist**. So lernt das neuronale Netzwerk, was es besser machen muss. Die Antworten eines neuronalen Netzwerks können in einer **Confusion Matrix** zusammengefasst werde. Das ist eine **Tabelle, die zeigt, wie oft ein neuronales Netzwerk etwas richtig oder falsch erkannt hat**. Daraus kann man dann die sogenannte **Accuracy** berechnen, also **wie genau das neuronale Netzwerk insgesamt war**.

In [ ]:
# --- Zweiter Code ---
from IPython.display import display
eval_button = widgets.Button(description="Bewerten")
eval_output = widgets.Output()

def predict(x, weights, biases):
    for i in range(len(weights)):
        act = 'sigmoid' if i == len(weights)-1 else 'relu'
        x, _ = dense_forward(x, weights[i], biases[i], act)
    return 1 if x[0] > 0.5 else 0

def on_eval_click(b):
    eval_output.clear_output()
    if not trained_model:
        with eval_output:
            print("Bitte zuerst das Modell trainieren.")
        return

    weights = trained_model["weights"]
    biases = trained_model["biases"]
    losses = trained_model["loss_trace"]

    ypred = [predict(xi, weights, biases) for xi in Xtest]
    TP = TN = FP = FN = 0
    for true, pred in zip(ytrue, ypred):
        if true == pred:
            if true == 1: TP += 1
            else: TN += 1
        else:
            if true == 1: FN += 1
            else: FP += 1
    acc = (TP + TN) / len(ytrue)

    with eval_output:
        plt.plot(losses)
        plt.title("Verlust-Funktion")
        plt.xlabel("Epoche")
        plt.ylabel("Verlust")
        plt.grid(True)
        plt.show()

        print("Confusion Matrix:")
        print(f"TN: {TN}  FP: {FP}")
        print(f"FN: {FN}  TP: {TP}")
        print(f"Genauigkeit: {acc:.2f}")

eval_button.on_click(on_eval_click)
display(widgets.VBox([eval_button, eval_output]))


**Prima**, jetzt hast Du ein eigenes neuronales Netzwerk **trainiert und bewertet**!